# SBML creator
`sbmlutils` provides helpers for the creation of SBML models from scratch.

## Create model
The easist way to create a new model is by using a dictionary of information for the model creation.

In [1]:
from sbmlutils.io import read_sbml, write_sbml, validate_sbml
from sbmlutils.creator import create_model

from sbmlutils.factory import *
from sbmlutils.units import *
from sbmlutils.metadata import *
from sbmlutils.metadata.sbo import *

import tempfile

In [2]:
# model definition
md = {
        "mid": "example_model",
        "model_units": ModelUnits(
            time="second",
            substance="mole",
            extent="mole",
            volume="litre",
        ),
        "compartments": [Compartment(sid="C", value=1.0)],
        "species": [
            Species(
                sid="S1",
                initialConcentration=10.0,
                compartment="C",
                hasOnlySubstanceUnits=False,
            ),
            Species(
                sid="S2",
                initialConcentration=0.0,
                compartment="C",
                hasOnlySubstanceUnits=False,
            )
        ],
        "parameters": [Parameter(sid="k1", value=1.0)],
        "reactions": [
            Reaction(sid="R1", equation="S1 -> S2", formula=("k1 * S1", None))
        ],
    }

# create the model as L3V1
with tempfile.TemporaryDirectory() as tmp_path:
    results = create_model(
        modules=md,
        output_dir=tmp_path,
        tmp=False,
        units_consistency=False,
        sbml_level=3,
        sbml_version=1,
    )
    # show level and version and print SBML
    doc = read_sbml(source=results.sbml_path, validate=False)
    sbml = write_sbml(doc)
    print(sbml)

'output_dir' should be a Path: /tmp/tmp40llfm00



--------------------------------------------------------------------------------
file:///tmp/tmp40llfm00/example_model.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:comp="http://www.sbml.org/sbml/level3/version1/comp/version1" level="3" version="1" comp:required="true">
  <model metaid="meta_example_model" id="example_model" name="example_model" substanceUnits="mole" timeUnits="second" volumeUnits="litre" extentUnits="mole">
    <listOfCompartments>
      <compartment id="C" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S1" compartment="C" initialConcentration="10" substanceUnits="mole" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="S2" compartment="C" initialConcen

## Units
It is highly recommended to annotate all units in the model. This allows automatic unit validation and unit conversions with the model.

As units one can either use:
- custom `UnitDefinitions`
- the `libsbml.UNIT_KIND_*` units such as `UNIT_KIND_SECOND` or `UNIT_KIND_LITRE`
- strings corresponding to the libsbml units

```
    ampere         farad  joule     lux     radian     volt
    avogadro       gram   katal     metre   second     watt
    becquerel      gray   kelvin    mole    siemens    weber
    candela        henry  kilogram  newton  sievert
    coulomb        hertz  litre     ohm     steradian
    dimensionless  item   lumen     pascal  tesla
```

## SBML Levels and Versions
Models can be generated in all SBML levels and versions via the `create_model` function.

In [3]:
from sbmlutils.io import read_sbml, write_sbml, validate_sbml
from sbmlutils.creator import create_model

from sbmlutils.factory import *
from sbmlutils.units import *
from sbmlutils.metadata import *
from sbmlutils.metadata.sbo import *

from libsbml import UNIT_KIND_SECOND, UNIT_KIND_MOLE, UNIT_KIND_METER, UNIT_KIND_LITER

import tempfile

In [4]:
# model definition
md = {
        "mid": "level_version",
        "model_units": ModelUnits(
            time=UNIT_KIND_SECOND,
            substance=UNIT_KIND_MOLE,
            extent=UNIT_KIND_MOLE,
            volume=UNIT_KIND_LITER,
        ),
        "compartments": [Compartment(sid="C", value=1.0)],
        "species": [
            Species(
                sid="S1",
                initialConcentration=10.0,
                compartment="C",
                hasOnlySubstanceUnits=False,
                boundaryCondition=True,
            )
        ],
        "parameters": [Parameter(sid="k1", value=1.0)],
        "reactions": [
            Reaction(sid="R1", equation="S1 ->", formula=("k1 * S1", None))
        ],
    }

supported_level_version = [
    (1, 1), (1, 2),
    (2, 1), (2, 2), (2, 3), (2, 4), (2, 5),
    (3, 1), (3, 2),
]

with tempfile.TemporaryDirectory() as tmp_path:

    for level, version in supported_level_version:
        # inject information
        md["mid"] = f"L{level}V{version}"
        results = create_model(
            modules=md,
            output_dir=tmp_path,
            tmp=False,
            units_consistency=False,
            sbml_level=level,
            sbml_version=version,
        )
        doc = read_sbml(source=results.sbml_path, validate=False)
        print(f"L{doc.getLevel()}V{doc.getVersion()}")
        if level == 3 and version == 1:
            sbml = write_sbml(doc)
            print("-" * 80)
            print(sbml)


'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m



--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L1V1.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L1V1

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L1V2.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L1V2

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L2V1.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L2V1

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L2V2.xml
valid                    : TRUE
check time (s)           : 0.001
-------------------

'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m
'output_dir' should be a Path: /tmp/tmpy4sxgm1m



--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L2V3.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L2V3

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L2V4.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L2V4

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L2V5.xml
valid                    : TRUE
check time (s)           : 0.001
--------------------------------------------------------------------------------

L2V5

--------------------------------------------------------------------------------
file:///tmp/tmpy4sxgm1m/L3V1.xml
valid                    : TRUE
check time (s)           : 0.001
-------------------

## Create FBA Model
This example demonstrates the creation of an SBML FBA model from scratch.

In [5]:
from sbmlutils.io import write_sbml, validate_sbml
from sbmlutils.creator import CoreModel

from sbmlutils.factory import *
from sbmlutils.units import *
from sbmlutils.metadata import *
from sbmlutils.metadata.sbo import *

### Model building
Creation of FBA model using multiple packages (`comp`, `fbc`).